In [2]:
pip install langchain_huggingface

  Obtaining dependency information for langchain_huggingface from https://files.pythonhosted.org/packages/81/ce/502157ef7390a31cc67e5873ad66e737a25d1d33fcf6936e5c9a0a451409/langchain_huggingface-1.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for huggingface-hub<1.0.0,>=0.33.4 from https://files.pythonhosted.org/packages/a8/af/48ac8483240de756d2438c380746e7130d1c6f75802ef22f3c6d49982787/huggingface_hub-0.36.2-py3-none-any.whl.metadata
  Obtaining dependency information for tokenizers<1.0.0,>=0.19.1 from https://files.pythonhosted.org/packages/2e/47/174dca0502ef88b28f1c9e06b73ce33500eedfac7a7692108aec220464e7/tokenizers-0.22.2-cp39-abi3-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/b5/36/7fb70f04bf00bc646cd5bb45aa9eddb15e19437a28b8fb2b4a5249fac770/filelock-3.20.3-py3-none-any.whl.metadata
  Obtaining dependency information for fsspec>=2023.5.0 from https://files.pythonhosted.org/packages/e6/

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_classic.prompts import PromptTemplate
from langchain_classic.chains import RetrievalQA

In [ ]:
## read pdf from us_census
loader = PyPDFDirectoryLoader("./us_census")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap = 200)

final_document = text_splitter.split_documents(documents=documents)
final_document[0]

Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 18.2 (Windows)', 'creationdate': '2023-09-09T07:52:17-04:00', 'author': 'U.S. Census Bureau', 'keywords': 'acsbr-015', 'moddate': '2023-09-12T14:44:47+01:00', 'title': 'Health Insurance Coverage Status and Type by Geography: 2021 and 2022', 'trapped': '/false', 'source': 'us_census/acsbr-015.pdf', 'total_pages': 18, 'page': 0, 'page_label': '1'}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015\nIssued September 2023\nDouglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to \nhealth coverage. For example, between 2021 and 2022, \nthe labor market continued to improve, which may \nhave affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, w

In [ ]:
len(final_document)

316

In [ ]:
## hugging face embeddings
huggingface_embeddings = HuggingFaceEmbeddings(
    model_name = "BAAI/bge-small-en-v1.5",
    model_kwargs = {'device' : 'cpu'},
    encode_kwargs = {'normalize_embeddings' : True}
)


In [ ]:
import numpy as np
array = np.array(huggingface_embeddings.embed_query(final_document[0].page_content))
array

array([-8.30602124e-02, -1.45066725e-02, -2.10276563e-02,  2.72682551e-02,
        4.53646928e-02,  5.28341383e-02, -2.53758840e-02,  3.61304283e-02,
       -9.08312276e-02, -2.77017690e-02,  7.97398165e-02,  6.42475337e-02,
       -3.54004763e-02, -4.04245183e-02, -1.13771278e-02,  4.45296392e-02,
       -3.88545846e-03, -3.79056227e-03, -4.54509966e-02,  2.67047361e-02,
       -2.05682125e-02,  2.87432224e-02, -2.41201594e-02, -3.69411856e-02,
        1.92781016e-02,  1.06194885e-02,  3.21827410e-03,  2.33256840e-03,
       -4.29321788e-02, -1.64999247e-01,  2.77008000e-03,  2.68277284e-02,
       -4.12894562e-02, -1.88446753e-02,  1.58918798e-02,  9.22327489e-03,
       -2.00688243e-02,  8.16561580e-02,  3.89413126e-02,  5.52223064e-02,
       -3.69984396e-02,  1.75319500e-02, -1.28966980e-02,  2.80628010e-04,
       -2.51580775e-02,  4.59334999e-03, -2.39578318e-02, -5.76557498e-03,
        6.02946477e-03, -3.61178443e-02,  3.84415947e-02, -1.75463501e-03,
        5.05655929e-02,  

In [ ]:
print(array.shape)

(384,)


In [ ]:
vectorstore = FAISS.from_documents(final_document[:120],huggingface_embeddings)
vectorstore

In [ ]:
## query search
query = "What is health insurance coverage?"
relevant_documents = vectorstore.similarity_search(query=query)
print(relevant_documents[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [ ]:
## retriever
retriever = vectorstore.as_retriever(search_type = "similarity",search_kwargs = {"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x131116990> search_kwargs={'k': 3}


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

hf_api_key = os.getenv("HUGGINGFACE_API_KEY")
if not hf_api_key:
    raise RuntimeError("HUGGINGFACE_API_KEY is not set. Add it to .env or export it in your shell.")
os.environ["HUGGINGFACE_API_KEY"] = hf_api_key

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.messages import HumanMessage

# Use Llama 3.1 - It is a primary production model for 2026
hf = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",
    huggingfacehub_api_token=hf_api_key,
    max_new_tokens=500,
)

chat_model = ChatHuggingFace(llm=hf)

query = "What is health insurance coverage?"
messages = [HumanMessage(content=query)]

# This should avoid the 410 error
response = chat_model.invoke(messages)
print(response.content)

Health insurance coverage refers to the financial protection provided to an individual or their family in the event of medical expenses or healthcare services. It is a type of insurance that helps pay for or reimburse medical costs incurred by the policyholder, either themselves or their dependents.

Health insurance coverage typically includes a range of benefits, such as:

1. **Medical expenses**: Coverage for doctor visits, hospital stays, surgeries, and other medical procedures.
2. **Preventive care**: Coverage for routine health screenings, vaccinations, and check-ups.
3. **Prescription medications**: Coverage for prescription medications, including brand-name and generic options.
4. **Emergency services**: Coverage for emergency room visits, ambulance services, and other urgent care services.
5. **Chronic condition management**: Coverage for ongoing treatment and management of chronic conditions, such as diabetes or asthma.
6. **Mental health services**: Coverage for mental healt

In [ ]:
# #Hugging Face models can be run locally through the HuggingFacePipeline class.
# from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

# hf = HuggingFacePipeline.from_model_id(
#     model_id="mistralai/Mistral-7B-v0.1",
#     task="text-generation",
#     pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
# )

# llm = hf 
# llm.invoke(query)

In [ ]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [ ]:
prompt = PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [ ]:
retrievalQA = RetrievalQA.from_chain_type(
    llm = hf,
    chain_type = 'stuff',
    retriever = retriever,
    return_source_documents = True,
    chain_type_kwargs = {"prompt" : prompt}
)

In [ ]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [ ]:
# call the QA
ans = retrievalQA.invoke({"query" : query})
print(ans['result'])

ValueError: Model Qwen/Qwen2.5-7B is not supported for task text-generation and provider featherless-ai. Supported task: conversational.

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_classic.chains import RetrievalQA

# 1. Use DeepSeek-R1 - It is the most active serverless model in 2026
# We force 'together' as the provider because it is the most stable for this ID
llm = HuggingFaceEndpoint(
    repo_id="deepseek-ai/DeepSeek-R1",
    huggingfacehub_api_token=hf_api_key,
    provider="together",  # This is the 'secret sauce' to avoid the 404/410
    max_new_tokens=512,
)

# 2. You MUST wrap it in ChatHuggingFace for RetrievalQA to work in 2026
chat_model = ChatHuggingFace(llm=llm)

# 3. Use the chat_model in your QA chain
retrievalQA = RetrievalQA.from_chain_type(
    llm=chat_model,
    chain_type="stuff",
    retriever=retriever,
)

# 4. Invoke
query = "DIFFERENCES IN THE UNINSURED RATE BY STATE IN 2022"
ans = retrievalQA.invoke({"query": query})
print(ans['result'])

<think>
Hmm, the user is asking about differences in the uninsured rate by state in 2022 based on the context provided. Let me go through the information carefully. 

The key points I can extract about state-level differences in 2022 are: Massachusetts had the lowest uninsured rate while Texas had the highest. This is explicitly stated in the context. 

The document also indicates that Medicaid expansion status correlated with coverage patterns. Expansion states had higher private and public coverage rates than nonexpansion states in both 2021 and 2022. Specifically, Medicaid coverage was 22.7% in expansion states versus 18.0% in nonexpansion states. 

Regarding geographic trends, Utah had the highest private coverage rate (tied with North Dakota) and lowest public coverage, while New Mexico showed the opposite pattern with highest public coverage and lowest private coverage. 

For year-over-year changes, only Maine saw an increase in uninsured rates (from 5.7% to 6.6%), while 27 state

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains import RetrievalQA

# 1. Take your exact prompt and put it in a HumanMessage
# We use HumanMessage because it's the most reliable way to send custom formatting to Instruct models
prompt = ChatPromptTemplate.from_template("""Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}

Question:{question}

Helpful Answers: """)

# 2. Re-initialize the RetrievalQA with this prompt
retrievalQA = RetrievalQA.from_chain_type(
    llm=chat_model, # Your working DeepSeek/Llama chat model
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt} # This injects your template
)

# 3. Call the QA
query = "DIFFERENCES IN THE UNINSURED RATE BY STATE IN 2022"
ans = retrievalQA.invoke({"query": query})
print(ans['result'])

<think>
Hmm, the user is asking about differences in the uninsured rate by state in 2022 based on the provided context. The context includes information about Medicaid expansion states versus non-expansion states, changes in uninsured rates from 2021 to 2022, and specific state examples.

Looking at the context, it mentions that Massachusetts had the lowest uninsured rate while Texas had the highest in 2022. The Medicaid expansion status appears to be a key factor - expansion states had higher coverage rates (both private and public) than non-expansion states. The context also notes that uninsured rates decreased in 27 states (mainly Medicaid expansion states) and only Maine had an increase.

For public versus private coverage patterns, Utah had the highest private coverage and lowest public coverage, while New Mexico showed the opposite pattern with highest public coverage and lowest private coverage. The document also references the District of Columbia but doesn't provide its specif